In [279]:
import os
import numpy as np
import glob
import re
import locale 
import pandas as pd 
import requests
import wget
import csv
import time
import datetime
import xlwings as xw
import openpyxl
from collections import defaultdict
import matplotlib.pyplot as plt 
import seaborn as sns #Visualiser avec seaborn

import lat_lon_parser #Convertir coordonnées


ModuleNotFoundError: No module named 'lat_lon_parser'

liens et répertoires

In [4]:
RankURL = "https://www.vendeeglobe.org/fr/classement"
DLRankURL = "https://www.vendeeglobe.org/download-race-data/vendeeglobe_"
SkippersURL = "https://www.vendeeglobe.org/fr/glossaire"


Web scrapping boats data 

In [136]:
def load_data():
    df=(is_downloadable('http:vendeeglobe.org/fr/classement/20201108_120200'))
    #df = pd.read_csv('vendeeglobe.org/fr/classement/20201108_120200')
    return df

In [137]:
#on check si le lien contient un paquet à télecharger
def is_downloadable(url):
    """
    Does the url contain a downloadable resource
    """
    h = requests.head(url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return True

Changement du format des data 

import xlwings
import glob

def Conv_format (file_name):
    excel_app=xlwings.App(visible=False)
    excel_book=excel_app.books.open(file)
    excel_book.save()
    excel_book.close()
    excel_app.quit()
    return True
    
for f in glob.glob("/Users/mec/Ludo-MEKONTCHOU/Projet final/data/*"):
    Conv_format(f)








contatenation des données 

In [ ]:
#Rajoutons la date et l'heure de la capture de d'information
files = glob.glob('/Users/mec/Ludo-MEKONTCHOU/Projet final/excels/*.xlsx')
concat=[]
for excel in files:
    temp=pd.read_excel(excel,header=3,nrows=33)
    temp["date_export"]=excel[-20:-5]
    temp.to_excel('excel')
   # concat.append(temp)

In [187]:
test[-20:-5]

'20201127_140000'

In [189]:
test='/Users/mec/Ludo-MEKONTCHOU/Projet final/excels/20201127_140000.xlsx'
temp=pd.read_excel(test,header=3,nrows=33)
temp["date_export"]=test[-20:-5]
temp
temp.to_excel('/Users/mec/Ludo-MEKONTCHOU/Projet final/excels/test.xlsx')

In [5]:
files = glob.glob('/Users/mec/Ludo-MEKONTCHOU/Projet final/excels/*.xlsx')
df=pd.concat(pd.read_excel(excel,header=3,nrows=33) for excel in files)
#usecols=("B:U")

In [8]:
#enregistrement de la concatenation pour le plus exécuter 
df.to_csv('/Users/mec/Ludo-MEKONTCHOU/Projet final/dataconcat/rawexport.csv')

#df2=pd.concat(pd.read_excel(excel, header=[0,1], skiprows=3, nrows=33) for excel in files)

#files = glob.glob('/Users/mec/Ludo-MEKONTCHOU/Projet final/excels/*.xlsx')
#df2=pd.concat(pd.read_excel(excel,header=[0,1],nrows=33, skiprows=3 for excel in files))

#data preparocessing 

In [11]:

df=df.dropna(axis=1,how="all")
#df.columns=[re.sub(r'\W+','',\
#            ' '.join([x.split("\n")[0] if x.split("\n")[0] and "Unnamed" not in x else "" if "Unnamed" in x else x for x in y])
#]
#df["natvoile"]=df["Unnamed: 2"]
#df[["nat","voile"]]=df["natvoile"].str.split(" ", expand=True)

df["nat"]=df["Unnamed: 2"].str.split(" ", expand=True)[0]
df["voile"]=df["Unnamed: 2"].str.split(" ", expand=True)[1]
df["skipper"]=df["Unnamed: 3"].str.split("\n").apply(lambda x: x[0] if isinstance(x,list) else x)
df["bateau"]=df["Unnamed: 3"].str.split("\n", expand=True)[1]
df["bateau"]=df["bateau"].str.lower()
df["heure"]=df["Heure FR\nHour FR"].str.split("\n", expand=True)[0]
df["heure"]=df["Heure FR\nHour FR"].str.split(" ").apply(lambda x: x[0] if isinstance(x,list) else x)

for col in df.columns :
    df[col]=df[col].str.strip()


In [12]:
#Suppression des valeurs nulles 
df=df.dropna(axis=1,how="all")

In [13]:
#remplacement des caracteres 
df.replace(r'\n', '', regex=True, inplace=True)

In [14]:
df.rename({'Unnamed: 1': 'Rank', 'nat': 'Country'}, axis=1, inplace=True)

In [52]:
df['bateau'] = df['bateau'].str.replace(' ', '').str.replace('[^a-zA-Z]', '').str.upper()

<ipython-input-52-1002f9bbd552>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['bateau'] = df['bateau'].str.replace(' ', '').str.replace('[^a-zA-Z]', '').str.upper()


In [55]:
df = df.rename(columns={'Heure FR\nHour FR': 'Heure FR'})

In [63]:
df.rename(columns=lambda x: x.split("\n")[0], inplace=True)


In [ ]:
df = df[df["Rank"].str.isnumeric()] 

#élimination des skippers éliminés ou ayant abandonné
#df["Rank"] = df["Rank"].astype(int) #pour visualisations

In [217]:
type(df["Rank"][0])

str

In [64]:
df.columns

Index(['Rank', 'Unnamed: 2', 'Unnamed: 3', 'Heure FR', 'Latitude', 'Longitude',
       'Cap', 'Vitesse', 'VMG', 'Distance', 'Cap', 'Vitesse', 'VMG',
       'Distance', 'Cap', 'Vitesse', 'VMG', 'Distance', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Au premier',
       'Unnamed: 13', 'Au précédent', 'Unnamed: 15', 'Unnamed: 18', 'Country',
       'voile', 'skipper', 'bateau', 'heure'],
      dtype='object')

In [97]:
df.iloc[13890,:].T

Rank                              RET
Unnamed: 2                     GBR 99
Unnamed: 3      Alex ThomsonHUGO BOSS
Heure FR                          NaN
Latitude                          NaN
Longitude                         NaN
Cap                               NaN
Vitesse                           NaN
VMG                               NaN
Distance                          NaN
Cap                               NaN
Vitesse                           NaN
VMG                               NaN
Distance                          NaN
Cap                               NaN
Vitesse                           NaN
VMG                               NaN
Distance                          NaN
Unnamed: 19                       NaN
Unnamed: 20                       NaN
Unnamed: 4                        NaN
Unnamed: 5                        NaN
Unnamed: 6                        NaN
arrivé                            NaN
duree course                      NaN
Unnamed: 9                        NaN
Unnamed: 10 

In [121]:
#fonction de conversion 
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [71]:
df.rename({'Unnamed: 7': 'arrivé', 
           'Unnamed: 8': 'duree course','Unnamed: 13': 'Au premier','Unnamed: 13':
           'Au précédent'}, axis=1, inplace=True)

In [ ]:
df.rename({'Unnamed: 18': 'orientation', 
           'Unnamed: 8': 'duree course','Unnamed: 13': 'Au premier','Unnamed: 13':
           'Au précédent'}, axis=1, inplace=True)

In [124]:
#on save le fichier en local 

df.to_csv('/Users/mec/Ludo-MEKONTCHOU/Projet final/dataconcat/dataclean.csv')

In [ ]:
#recuperation des spec bateau

soup = BeautifulSoup(requests.get("https://www.vendeeglobe.org/fr/glossaire").content.decode("utf-8"))

dat=list()
for boat in soup.find_all("div", {"class":"boats-list__popup-infos"}):
    dict_ = {"nom bateau ":boat.find("h3").text}
    for li in boat.find_all("li")[:-1]:
        dict_[li.text.split(":")[0].strip()] = li.text.split(":")[1].strip()
    dat.append(dict_)

df_spec = pd.DataFrame(dat)
for col in df_spec.columns:
    df_spec[col] = df_spec[col].str.lower()
    


In [36]:
df_spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   nom bateau                 33 non-null     object
 1   Numéro de voile            32 non-null     object
 2   Anciens noms du bateau     33 non-null     object
 3   Architecte                 33 non-null     object
 4   Chantier                   33 non-null     object
 5   Date de lancement          33 non-null     object
 6   Longueur                   33 non-null     object
 7   Largeur                    33 non-null     object
 8   Tirant d'eau               33 non-null     object
 9   Déplacement (poids)        33 non-null     object
 10  Nombre de dérives          33 non-null     object
 11  Hauteur mât                33 non-null     object
 12  Voile quille               32 non-null     object
 13  Surface de voiles au près  33 non-null     object
dtypes: object(14

In [27]:
#Copie locale du fichier des spec 
df_spec.to_csv('/Users/mec/Ludo-MEKONTCHOU/Projet final/dataconcat/boatrawexport.csv')

In [35]:
type(df_spec['Anciens noms du bateau'][0])

list

In [41]:
#data preprocessing 



#conversion des données en int/float
num_cols = ['Longueur', 'Largeur',"Tirant d'eau" ,'Déplacement (poids)','Hauteur mât','Surface de voiles au près']
df_spec[num_cols] = df_spec[num_cols].apply(lambda x: x.str.replace(re.compile('m2|m²|m|t[a-z]*|nc|NC'), '').str.replace(',', '.')).apply(pd.to_numeric)


In [42]:
df_spec.rename(columns=lambda x: x.strip(), inplace=True)

In [43]:
df_boats["Numéro de voile"] = df_boats["Numéro de voile"].fillna(59)

In [46]:
df_spec[["Anciens noms du bateau", "Voile quille"]] = df_spec[["Anciens noms du bateau", "Voile quille"]].replace(np.nan,'')


In [48]:
df_spec[["Nombre de dérives"]] = df_spec[["Nombre de dérives"]].replace(["foiler", "2 asymétriques"],['foils', "2"])

In [47]:
df_spec['nom bateau'] = df_spec['nom bateau'].str.replace(' ', '').str.replace('[^a-zA-Z]', '').str.upper()

<ipython-input-47-99acc60d7348>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_spec['nom bateau'] = df_spec['nom bateau'].str.replace(' ', '').str.replace('[^a-zA-Z]', '').str.upper()


In [ ]:
df_spec["Annee"] = pd.to_datetime(df_spec["Date de lancement"].apply(lambda x: x[-4:]), format="%Y").dt.year


In [113]:
df_spec.rename({"Annee": 'annee_bateau'
           }, axis=1, inplace=True)

In [102]:
df_spec = df_spec.drop("Date de lancement",1)

In [104]:
dict_ = {"foiler":"foils","2":"no foils","2 asymétriques":"no foils","foils":"foils"}
df_spec["Foils"] = df_spec["Nombre de dérives"].map(dict_)
df_spec = df_spec.drop("Nombre de dérives",1)

In [114]:
 df_spec.head()

,nom bateau,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Hauteur mât,Voile quille,Surface de voiles au près,annee_bateau,Foils
0,NEWRESTARTFENTRES,fra 56,"[no way back, vento di sardegna]",vplp/verdier,persico marine,18.28,5.85,4.5,7.0,29.0,monotype,320,2015,foils
1,PUREBESTWESTERN,fra 49,"[gitana eighty, synerciel, newrest-matmut]",bruce farr design,southern ocean marine (nouvelle zélande),18.28,5.80,4.5,9.0,28.0,acier forgé,280,2007,no foils
2,TSEMYPLANET,fra72,"[famille mary-etamine du lys, initiatives coe...",marc lombard,mag france,18.28,5.54,4.5,9.0,29.0,acier,260,1998,no foils
3,MATRECOQIV,17,[safran 2 - des voiles et vous],verdier - vplp,cdk technologies,18.28,5.80,4.5,8.0,29.0,acier mécano soudé,310,2015,foils
4,CHARAL,08,[],vplp,cdk technologies,18.28,5.85,4.5,8.0,29.0,acier,320,2018,foils


In [115]:
#Copie locale du fichier des spec 
df_spec.to_csv('/Users/mec/Ludo-MEKONTCHOU/Projet final/dataconcat/boatclean.csv')

In [134]:
df_recup=pd.read_csv('/Users/mec/Ludo-MEKONTCHOU/Projet final/dataconcat/dataclean.csv')
df=df_recup


In [308]:
#merge des deux bases
df_merged = pd.merge(df_recup,
              df_spec,
              left_on='bateau',
              right_on='nom bateau',
              how='left')
df_merged.to_csv('/Users/mec/Ludo-MEKONTCHOU/Projet final/dataconcat/data_merged.csv')

In [309]:
df_merged.head()

,Unnamed: 0,Rank,Unnamed: 2,Unnamed: 3,Heure FR,Latitude,Longitude,Cap,Vitesse,VMG,...,Chantier,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Hauteur mât,Voile quille,Surface de voiles au près,annee_bateau,Foils
0,0,1,FRA 79,Charlie DalinAPIVIA,14:30 FR,41°31.42'S,09°11.14'W,68°,19.5 kts,14.6 kts,...,cdk technologies - merconcept,18.28,5.85,4.5,8.0,29.0,acier,350.0,2019.0,foils
1,1,2,FRA 59,Thomas RuyantLinkedOut,14:30 FR,38°58.44'S,14°54.88'W,181°,13.5 kts,5.5 kts,...,persico,18.28,5.85,4.5,8.0,29.0,acier forgé,350.0,2019.0,foils
2,2,3,FRA 01,Jean Le CamYes we Cam !,14:30 FR,36°17.48'S,14°48.81'W,110°,17.1 kts,16.9 kts,...,cdk technologies,18.28,5.90,4.5,8.0,28.0,acier forgé,300.0,2007.0,no foils
3,3,4,FRA 17,Yannick BestavenMaître Coq IV,14:30 FR,38°01.60'S,18°23.24'W,105°,16.1 kts,15.8 kts,...,cdk technologies,18.28,5.80,4.5,8.0,29.0,acier mécano soudé,310.0,2015.0,foils
4,4,5,FRA 85,Kevin EscoffierPRB,14:30 FR,37°55.75'S,18°38.56'W,107°,17.1 kts,16.9 kts,...,cdk technologies,18.28,5.50,4.5,NaN,27.4,acier mécano soudé,300.0,2010.0,foils


le plus proche du premier 

prevision des positions en multivariés

Exploitation des données 

Durée du trajet sur le rang 

In [320]:
df_merged.head().T

,0,1,2,3,4
Rank,1,2,3,4,5
Heure FR,14:30 FR,14:30 FR,14:30 FR,14:30 FR,14:30 FR
Latitude,41°31.42'S,38°58.44'S,36°17.48'S,38°01.60'S,37°55.75'S
Longitude,09°11.14'W,14°54.88'W,14°48.81'W,18°23.24'W,18°38.56'W
Cap_30min,68°,181°,110°,105°,107°
Vitesse_30min,19.5 kts,13.5 kts,17.1 kts,16.1 kts,17.1 kts
VMG,14.6 kts,5.5 kts,16.9 kts,15.8 kts,16.9 kts
Distance_30min,9.7 nm,6.8 nm,8.5 nm,8.1 nm,8.6 nm
Cap_lastcla,76°,182°,119°,112°,114°
Vitesse_lastclass,21.3 kts,12.8 kts,16.5 kts,17.5 kts,17.2 kts


In [311]:
df_merged.rename({"Vitesse": 'Vitesse_30min',"Vitesse.1": 'Vitesse_lastclass',"Vitesse.2": 'Vitesse_24H',
                  "Distance": 'Distance_30min',"Distance.1": 'Distance_lastclass',"Distance.2": 'Distance_24H',
                  "Unnamed: 19": 'DTF',"Unnamed: 20": 'DTL',"Cap": 'Cap_30min',"Cap.1": 'Cap_lastcla',"Cap.2": 'Cap_24H',
           }, axis=1, inplace=True)

In [306]:
#df_merged = df_merged.dropna()

In [312]:
df_merged =df_merged.drop("Unnamed: 2",1)
df_merged =df_merged.drop("Unnamed: 0",1)
df_merged =df_merged.drop("Unnamed: 3",1)

Tableau de fin de course avec les vitesses moyennes et distances moyennes 

In [299]:
# we convert all the numerical values into floats
#df_merged['Rank'] =df_merged['Rank'].apply(int)
df_merged['Rank'] = pd.to_numeric(
    df_merged['Rank'], 
    errors='coerce'
).fillna(0).astype('int')  

In [284]:
df_test=df_merged
df_test['Vitesse_24H'] = pd.to_numeric(
   df_test['Vitesse_24H'], 
    errors='coerce'
).fillna(0).astype('int') 

In [265]:
df_merged['Distance_24H'] = pd.to_numeric(
    df_merged['Distance_24H'], 
    errors='coerce'
).fillna(0).astype('int') 

In [266]:
df_merged['DTF'] = pd.to_numeric(
    df_merged['DTF'], 
    errors='coerce'
).fillna(0).astype('int') 

In [267]:
df_merged['DTL'] = pd.to_numeric(
    df_merged['DTL'], 
    errors='coerce'
).fillna(0).astype('int')

In [314]:
df_merged.columns

Index(['Rank', 'Heure FR', 'Latitude', 'Longitude', 'Cap_30min',
       'Vitesse_30min', 'VMG', 'Distance_30min', 'Cap_lastcla',
       'Vitesse_lastclass', 'VMG.1', 'Distance_lastclass', 'Cap_24H',
       'Vitesse_24H', 'VMG.2', 'Distance_24H', 'DTF', 'DTL', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'arrivé', 'duree course', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Au premier', 'Au précédent',
       'Au précédent.1', 'Unnamed: 15', 'Unnamed: 18', 'Country', 'voile',
       'skipper', 'bateau', 'heure', 'nom bateau', 'Numéro de voile',
       'Anciens noms du bateau', 'Architecte', 'Chantier', 'Longueur',
       'Largeur', 'Tirant d'eau', 'Déplacement (poids)', 'Hauteur mât',
       'Voile quille', 'Surface de voiles au près', 'annee_bateau', 'Foils'],
      dtype='object')

In [ ]:
numerical_cols = ['Cap_30min',
       'Vitesse_30min', 'VMG', 'Distance_30min', 'Cap_lastcla',
       'Vitesse_lastclass', 'VMG.1', 'Distance_lastclass', 'Cap_24H',
       'Vitesse_24H', 'VMG.2', 'Distance_24H', 'DTF', 'DTL']
df_merged[numerical_cols] = df_merged[numerical_cols].apply(lambda x: x.str.replace(re.compile('[a-z]+|°'), '')).apply(pd.to_numeric(errors='coerce'))


In [317]:
df_vit_dist = pd.DataFrame(df_merged.groupby(['skipper', 'Foils', 'Longueur', 'Largeur', "Tirant d'eau",
                                     'Déplacement (poids)', 'Hauteur mât',
                                     'Surface de voiles au près'])['Vitesse_24H'])
df_vit_dist.reset_index(inplace=True)





Vitesse sur le rang 

Année du bateau 

In [ ]:
Expérience du pilote sur le rang 

prevision des positions en multivariés

In [ ]:
maintenir l'écart au premier

Correlation vitesse et autres caracteristiques 

In [ ]:

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

fig.suptitle('speed in function of some boats caracteristics')

sns.barplot(ax=axes[0, 0], data=df_vit_dist, x="Foils", y='Vitesse_24H', ci=None)
sns.barplot(ax=axes[0, 1], data=df_vit_dist, x="Largeur", y='Vitesse_24H', hue="Foils", ci=None)
sns.barplot(ax=axes[0, 2], data=df_vit_dist, x="Hauteur mât", y='Vitesse_24H', hue="Foils", ci=None)
sns.barplot(ax=axes[1, 0], data=df_vit_dist, x="Surface de voiles au près", y='Vitesse_24H', hue="Foils", ci=None)
sns.barplot(ax=axes[1, 1], data=df_vit_dist, x="DTF", y='Vitesse_24H', hue="Foils", ci=None)
sns.barplot(ax=axes[1, 2], data=df_vit_dist, x="Déplacement (poids)", y='Vitesse_24H', hue="Foils", ci=None)

plt.show()


In [290]:
df_merged['Vitesse_24H'].mean()

0.0